In [ ]:
from deepeval.test_case.llm_test_case import LLMTestCase, LLMTestCaseParams
from deepeval.test_case.conversational_test_case import ConversationalTestCase
from deepeval.metrics.conversational_g_eval.conversational_g_eval import ConversationalGEval
import deepeval.models.llms.openai_model as deepeval_models
from deepeval import evaluate
gpt_41_mini = deepeval_models.GPTModel(
    model="gpt-4.1-mini",
    timeout=60,
)
json_file = 'conversation_tone.json'

In [ ]:
import json
from deepeval.test_case.llm_test_case import LLMTestCase
from deepeval.test_case.conversational_test_case import ConversationalTestCase
from typing import List
def load_testcase_from_json(json_file) -> List[ConversationalTestCase]:
    try:
        with open( json_file,"r") as f:
            parsed_data = json.load(f)
        deepeval_test_cases = []
        for conv_data in parsed_data:
            llm_turns = []
            for turn_data in conv_data.get("turns", []):
                user_input = turn_data.get("user_input")
                gender_context_val = turn_data.get("user_gender_context")
                actual_output_placeholder = turn_data.get("bot_actual_output_placeholder", "")  
                current_context = []
                if gender_context_val and gender_context_val != "null":  #Kiểm tra null dưới dạng chuỗi
                    current_context.append(f"User gender: {gender_context_val}")
            
                if user_input:  #Chỉ thêm turn nếu có user_input
                    llm_turns.append(
                        LLMTestCase(
                            input=user_input,
                            actual_output=actual_output_placeholder, # Sẽ được điền sau
                            context=current_context if current_context else None  # DeepEval có thể muốn None nếu context rỗng
                        )
                    )
            if llm_turns:  #Chỉ thêm ConversationalTestCase nếu có turns hợp lệ
                deepeval_test_cases.append(ConversationalTestCase(turns=llm_turns))  #DeepEval v0.20+ dùng messages
                                                                                    #(hoặc `turns=llm_turns` cho phiên bản cũ hơn)
        print(f"Đã load thành công {len(deepeval_test_cases)} ConversationalTestCase objects.")
        return deepeval_test_cases
    except json.JSONDecodeError as e:
        print(f"Lỗi giải mã JSON: {e}")
    except Exception as e:
        print(f"Lỗi không xác định khi xử lý: {e}")

In [ ]:
from repositories.user import create as create_user, CreateUserModel
from repositories.thread import create as create_thread, CreateThreadModel
from service.store_chatbot_v2 import gen_answer, ConfigModel
from uuid import uuid4
from models.user import UserRole    
import service.openai as openai_service
from openai.types.chat_model import ChatModel

def get_actual_answer(input: str, gender_context: str, response_model: str, use_fine_tune_tone: bool) -> str:
    user = create_user(
        CreateUserModel(user_name=str(uuid4()), role=UserRole.chainlit_user)
    )
    thread = create_thread(CreateThreadModel(user_id=user.id, name=user.user_name))
    config = ConfigModel(
        response=response_model,
        use_fine_tune_tone=use_fine_tune_tone,
    )
    history = [
        {"role": "user", "content": str(input)}
    ]

    return gen_answer(
        thread_id=thread.id,
        history=history,
        user_id=user.id,
    )

def lst_context_to_str(lst: list) -> str:
    turn_str = ""
    for t in lst:
        turn_str += t
    return turn_str


def get_actual_answer_for_all_testcase_without_conversation_history(json_file:str,gen_answer_model:ChatModel=None,iter_checkpoint:int=0):
    #deepeval_test_cases = load_testcase_from_json(json_file) # load test case with actual_output werent generated
    '''
    get_actual_answer for all testcase
    iter_checkpoint is the index of the checkpoint fine-tuned model
    gen_answer_model is the fine-tuned model to use for generating responses
    '''
    conversations = load_testcase_from_json(json_file)
    process_conversations = []
    id = 0
    for conversational_test_case in conversations:
        processed_turns = []
        for turn in conversational_test_case.turns:
            turn.actual_output = get_actual_answer(turn.input, lst_context_to_str(turn.context),gen_answer_model)
            print(turn.input,turn.actual_output)
            processed_turn = {
                "user_input": turn.input,
                "user_gender_context": turn.context,
                "bot_actual_output_placeholder": turn.actual_output
            }
            processed_turns.append(processed_turn)

        process_conversations.append({
            "conversation_id": conversational_test_case.id,
            "turns": processed_turns
        })
        id += 1
        '''write to json file'''
        print(f"Processed {id} conversations with actual answers.")
    with open(f"conversation_tone_{iter_checkpoint}.json", "w") as f:
        json.dump(process_conversations, f)

def get_actual_answer_with_conversation_history(json_file: str, gen_answer_model:ChatModel=None,iter_checkpoint:int=0):
    
    conversations = load_testcase_from_json(json_file)
    process_conversations = []
    id = 0
    openai_service._fine_tuning_model = gen_answer_model # cap nhat tung checkpoint model cho gen_answer

    for conv in conversations:
        processed_turns = []
        turns = conv.turns
        user = create_user(
            CreateUserModel(user_name=str(uuid4()), role=UserRole.chainlit_user)
        )
        thread = create_thread(CreateThreadModel(user_id=user.id, name=user.user_name))
    
        # Initialize conversation history with system message
        history = [
            {"role": "system", "content": "##BASE KNOWLEDGE:\n" + turns[0]["user_gender_context"]}
        ]
        
        for turn in turns:
            # Add user input to history
            history.append({"role": "user", "content": turn["user_input"]})
            
            # Set the model for this conversation
            #openai_service._fine_tuning_model = gen_answer_model
            
            # Generate bot response using accumulated history
            bot_response = gen_answer(
                #thread_id=uuid4(),  # Generate new thread ID for each turn
                thread_id=thread.id,
                history=history,
                #user_id=uuid4(),  # Generate new user ID for each turn
                user_id=user.id,
            )
            turn.actual_output = bot_response
            # Add bot response to history for next turn
            history.append({"role": "assistant", "content": bot_response})
            
            # Store processed turn
            
            processed_turn = {
                "user_input": turn.input,
                "user_gender_context": turn.context,
                "bot_actual_output_placeholder": turn.actual_output
            }
            processed_turns.append(processed_turn)
        
        process_conversations.append({
            "conversation_id": conv.id,
            "turns": processed_turns
        })
        id += 1
        print(f"Processed {id} conversations with actual answers.")
    with open(f"{json_file}_{iter_checkpoint}.json", "w") as f:
        json.dump(process_conversations, f)

In [ ]:

pronoun_consistency_metric = ConversationalGEval(
    name="Vietnamese Pronoun Consistency",
    criteria="""Đánh giá khả năng của chatbot trong việc sử dụng đại từ nhân xưng tiếng Việt một cách chính xác và nhất quán trong suốt cuộc hội thoại. Cụ thể:
    1. Chatbot (assistant) trong 'actual_output' phải LUÔN LUÔN tự xưng là 'em'.  Ví dụ: 'dạ em chào anh', 'em có thể giúp gì ạ'. KHÔNG được dùng 'tôi', 'mình'
    2. Cách chatbot gọi người dùng (user) phải dựa trên thông tin được cung cấp hoặc cách người dùng tự xưng:
        - Nếu 'User gender' được cung cấp là 'male' hoặc người dùng tự xưng là 'anh' như 'anh muốn hỏi...', chatbot phải gọi người dùng là 'anh'.
        - Nếu 'User gender' được cung cấp là 'female' hoặc người dùng tự xưng là 'chị' như 'chị muốn hỏi...', chatbot phải gọi người dùng là 'chị'.
        - Nếu người dùng tự xưng là 'chú', chatbot phải gọi người dùng là 'chú' và tự xưng 'cháu'.
        - Nếu người dùng tự xưng là 'bác', chatbot phải gọi người dùng là 'bác' và tự xưng 'cháu'.
        - Nếu người dùng tự xưng là 'cô', chatbot phải gọi người dùng là 'cô' và tự xưng 'cháu'. 
        - Nếu 'User gender' được cung cấp là 'unknown' hoặc không được cung cấp, và người dùng không tự xưng theo một đại từ cụ thể nào ở trên, chatbot phải gọi người dùng là 'anh/chị'.

    3. Tính nhất quán: Chatbot phải duy trì cách xưng hô đã được thiết lập với người dùng một cách nhất quán trong các lượt trả lời tiếp theo trong cùng một cuộc hội thoại, trừ khi có thông tin mới rõ ràng thay đổi cách xưng hô.
    4. Không được sử dụng các cách xưng hô không phù hợp hoặc thiếu tôn trọng.
      HƯỚNG DẪN CHẤM ĐIỂM:
    - Điểm 1.0: Tuân thủ hoàn hảo tất cả các quy tắc trên trong mọi lượt của hội thoại.
    - Phạt nặng (điểm gần 0): Nếu chatbot tự xưng sai (ví dụ: xưng 'tôi' thay vì 'em').
    - Phạt nặng (điểm gần 0): Nếu chatbot gọi sai người dùng một cách rõ ràng (ví dụ: context là 'User gender: male' nhưng chatbot gọi là 'chị').
    - Xem xét toàn bộ cuộc hội thoại để đánh giá tính nhất quán.
    """,
    
    evaluation_params=[
        LLMTestCaseParams.INPUT,            # Để phân tích input của user (ví dụ: "anh muốn hỏi...")
        LLMTestCaseParams.ACTUAL_OUTPUT,    # Để phân tích output của model
        LLMTestCaseParams.CONTEXT,        # Nếu thông tin `User gender` được truyền qua context cho mỗi lượt
    ],
    model=gpt_41_mini,
    
)

In [ ]:
test_case = ConversationalTestCase(
    turns=[
        LLMTestCase(
            input="Chị muốn hỏi cách tra cứu điểm mua hàng đã tích được tại FPT Shop .",
            actual_output= " Chị có thể thực hiện tra cứu điểm tích [tại đây](https://fptshop.com.vn/tai-khoan/lich-su-tich-diem) bằng cách đăng nhập số điện thoại mua hàng của chị.",
            # Không cần context nếu input đã đủ rõ, nhưng criteria cần xử lý được
            # context=["User gender: female"] # Có thể thêm nếu muốn rõ ràng hơn
            context=["User gender: unknown"]
        ),
        LLMTestCase(
            input="Vậy còn tra cứu về thông tin trúng thưởng của FPT Shop khi tham gia các chương trình mini game?",
            actual_output="Chị có thể thực hiện tra cứu [tại đây](https://fptshop.com.vn/khuyen-mai/thong-tin-trao-thuong)",
            context=["User gender: unknown"]
        )
    ]
)
pronoun_consistency_metric.measure(test_case)
print(pronoun_consistency_metric.score, pronoun_consistency_metric.reason)


In [ ]:
test_case1 = ConversationalTestCase(
    turns=[
        LLMTestCase(
            input="Chị muốn hỏi cách tra cứu điểm mua hàng đã tích được tại FPT Shop .",
            actual_output= " Chị có thể thực hiện tra cứu điểm tích [tại đây](https://fptshop.com.vn/tai-khoan/lich-su-tich-diem) bằng cách đăng nhập số điện thoại mua hàng của chị.Chị cần em hỗ trợ gì nữa không ạ?",
            # Không cần context nếu input đã đủ rõ, nhưng criteria cần xử lý được
            # context=["User gender: female"] # Có thể thêm nếu muốn rõ ràng hơn
            context=["User gender: unknown"]
        ),
        LLMTestCase(
            input="Vậy còn tra cứu về thông tin trúng thưởng của FPT Shop khi tham gia các chương trình mini game?",
            actual_output="Chị có thể thực hiện tra cứu [tại đây](https://fptshop.com.vn/khuyen-mai/thong-tin-trao-thuong). Chị cần em hỗ trợ gì nữa không ạ?",
            context=["User gender: unknown"]
        )
    ]
)
pronoun_consistency_metric.measure(test_case1)
print(pronoun_consistency_metric.score, pronoun_consistency_metric.reason)

In [ ]:
# danh gia rieng le 1 testcase
pronoun_consistency_metric.measure(tc)
print(pronoun_consistency_metric.score, pronoun_consistency_metric.reason)

In [ ]:

        

'''save the output of get_actual_answer for all testcase to json file for each model'''


def eval_tc(_testcases,_metrics,iter_checkpoint):
    evaluation_results = evaluate(
        test_cases=_testcases,
        metrics=_metrics,
    )
    # điểm số của metric được lưu trong mỗi đối tượng test case sau khi evaluate
    score = 0
    for result in evaluation_results: # evaluate() trả về list các test cases đã được cập nhật
        print(f"Test Case ID: {result.id if result.id else 'N/A'}") # ConversationalTestCase có thể không có id trừ khi bạn set
        for metric_result in result.metrics: # Mỗi test case có thể có nhiều metric
            if metric_result.name == pronoun_consistency_metric.name:
                print(f"  Metric: {metric_result.name}")
                print(f"  Score: {metric_result.score}")
                if metric_result.reason: 
                    print(f"  Reason: {metric_result.reason[:300]}...") # In một phần lý do
                print("-" * 20)
                score += metric_result.score
    mean_score = score/len(evaluation_results)
    print(f"Mean score: {mean_score}")
            
    
    # get the mean score of all testcase
    
    #mean_score = evaluation_results.score.mean()
    # save result to file
    with open(f"evaluation_results_{iter_checkpoint}.json", "w") as f:
        json.dump(evaluation_results, f)
        json.dump(mean_score, f)
    
    




In [ ]:
checkpoint_lst = ["ft:gpt-4o-mini-2024-07-18:personal::Bdao6dn4:ckpt-step-50","ft:gpt-4o-mini-2024-07-18:personal::Bdao6ShS:ckpt-step-100","ft:gpt-4o-mini-2024-07-18:personal::Bdao7Wp2"]

def get_actual_answer_for_all_testcase_for_all_checkpoint(checkpoint_lst):
    for iter_checkpoint,checkpoint_model in enumerate(checkpoint_lst):
        print(checkpoint_model)
        # load checkpoint
        model = checkpoint_model

        # 1 trong 2 cach de luu actual_output vao json file moi
        #get_actual_answer_for_all_testcase_without_conversation_history(json_file,model,iter_checkpoint)
        get_actual_answer_with_conversation_history(json_file,model,iter_checkpoint)





In [ ]:
get_actual_answer_for_all_testcase_for_all_checkpoint(checkpoint_lst)

In [ ]:
"""Thay vi chi dùng checkpoint cuoi của fine-tuned model , dùng toàn bộ checkpoint để đánh giá"""
def eval_all_checkpoint(deepeval_test_cases,checkpoint_lst):
    for iter_checkpoint,checkpoint_model in enumerate(checkpoint_lst):
        load_testcase_from_json(f'{json_file}_{iter_checkpoint}.json')
        print(checkpoint_model)
        # load checkpoint
        # evaluate
        eval_tc(deepeval_test_cases,[pronoun_consistency_metric],iter_checkpoint)